In [16]:
import openai
from dotenv import load_dotenv
from github import Github, GithubException
import re

In [17]:
def parse_python_code(input_str):
    pattern = r'```python\n(.*?)\n```'
    result = re.search(pattern, input_str, re.DOTALL)
    if result:
        return result.group(1)
    else:
        return "No match found"

In [18]:
load_dotenv()

True

In [19]:
# Login and get repo
repo_path = "Scott-Huston/gpt_dev"

g = Github(os.getenv("GH_TOKEN"))
repo = g.get_repo(repo_path)

In [20]:
# Get file paths for all files in repo
# get_contents only gets root level at first

def get_repo_filepaths(repo):
    filepaths = []
    contents = repo.get_contents("")
    while contents:
        file_content = contents.pop(0)
        if file_content.type == "dir":
            # Add contents of subfolder
            contents.extend(repo.get_contents(file_content.path))
        else:
            if file_content.path != ".gitignore": # gitignore info just unnecessary tokens
                filepaths.append(file_content.path)
    return filepaths

filepaths

['README.md', 'requirements.txt', 'test.ipynb']

In [21]:
# Represent contents of repo in a long string
repo_content_string = ""

for filepath in filepaths:
    repo_content_string += f"FILE PATH:\n{filepath}\nFILE CONTENT:\n"

    file_content = repo.get_contents(filepath).decoded_content
    repo_content_string += f"{file_content}\n\n"

repo_content_string

'FILE PATH:\nREADME.md\nFILE CONTENT:\nb\'# gpt_dev\\n\\npip install -r requirements.txt\\n\\nV1 Overall flow:\\n1. User inputs link to repo and prompt/documentation/ticket/coding standards etc.\\n2. Model asks any clarifying questions\\n3. Model creates PR\\n    (maybe another call to assess / edit the PR? Does the quality go up with a bunch of calls?)\\n4. Optional loop of\\n    a. additional feedback\\n    b. updated PR\\n\\nMore granular view\\n\\n- (Done) Represent entire repo as block of text. Possibly this can be done better with sequential calls to the model summarizing relevant parts of each file or something\\n    - Github api to iterate through all files and get contents?\\n    - How does the repo chat thing that exists do this?\\n\\n- How do I know whether there should be a message from the user with clarifications, or straight to pr gen?\\n\\n- How to create PR? How to specify content of different files? Need to give the model the github api docs?\\n    \'\n\nFILE PATH:\nr

In [22]:
# Authenticate
openai.api_key = os.getenv("OPENAI_API_KEY")

In [47]:
import prompts
print(prompts.SYSTEM_MESSAGE)

You are a world-class 10X programmer.


In [48]:
# system_message = "You are a world-class 10X programmer." 


task_description = "The idea of the test.ipynb notebook is to have a user describe a pull request on a repo they would \
like to see and have gpt-4 (a human-level language model) generate code to create the pull request. \
Please take the general concept of the code in the test.ipynb notebook and turn it into well \
structured code with a script that can be executed in a user-friendly manner from the command line. Be sure to update \
the readme with good documentation and instructions as well. You should create a Python script from the test.ipynb notebook \
contents that can be run from the command line. There is no specific file structure or naming convention you need to use. \
Please keep the existing code in the test.ipynb notebook as is. Allow users to specify any useful user inputs through \
command-line arguments. Continue to use the gpt-4 model."

clarification_prompt = "Do not attempt to complete the task yet. Are there any questions you have about the task before you begin?"

first_message = f"""Here are the contents of a code repository on Github:
{repo_content_string}
Task Description:
{task_description}
{prompts.CLARIFICATION_PROMPT}
"""

# response = openai.ChatCompletion.create(
#   model="gpt-4",
#   messages=[
#         {"role": "system", "content": prompts.SYSTEM_MESSAGE},
#         {"role": "user", "content": first_message}
#     ]
# )

In [23]:
len(first_message.split(" "))

6522

In [38]:
response

<OpenAIObject chat.completion id=chatcmpl-7LEZOkULsEVppesTFu4VuLDinblbI at 0x128877e90> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Before I begin converting the code in the test.ipynb notebook to a script that can be executed from the command line, I have a few questions:\n\n1. Do you want me to create a separate script file for the command-line execution, or should I incorporate it into an existing script file?\n2. Are there any specific command-line arguments you would like to include? For example, would you like to allow users to specify the GitHub repository link or authentication token through command-line arguments?\n3. The current code uses gpt-4 as the language model. Do you want me to keep using it or switch to a different model?",
        "role": "assistant"
      }
    }
  ],
  "created": 1685295250,
  "id": "chatcmpl-7LEZOkULsEVppesTFu4VuLDinblbI",
  "model": "gpt-4-0314",
  "object": "chat.completion

In [40]:
first_message = f"""Here are the contents of a code repository on Github:
{repo_content_string}
Task Description:
{task_description}\n\n"""

In [52]:
repo.full_name

'Scott-Huston/gpt_dev'

In [41]:
create_pr_message = f"""Ok, now please create a pull request to complete the task using the PyGithub Python library. You should create a new branch, create/modify/delete files as necessary, and then create a pull request.

# Example of creating a branch

source_branch = '{repo.default_branch}'
target_branch = 'test'

try:
    # Test to see if branch already exists
    repo.get_branch("test")
except GithubException as e:
    if e.status == 404:
        # If branch does not exist, create it
        sb = repo.get_branch(source_branch)
        repo.create_git_ref(ref='refs/heads/' + target_branch, sha=sb.commit.sha)

# Example of creating a new file
file_content = "This is the file content"
commit_message = "create test file"
repo.create_file(path="test.txt", message=commit_message, content=file_content, branch="test")

# Example of updating a file
contents = repo.get_contents("test.txt", ref="test")
repo.update_file(contents.path, "more tests", "more tests", contents.sha, branch="test")

# Example of deleting a file
contents = repo.get_contents("test.txt", ref="test")
repo.delete_file(contents.path, message="remove test", sha=contents.sha, branch="test")

# Example of creating the PR
body = '''
This PR created, updated and deleted a test file
'''
repo.create_pull(title="Test file", body=body, head="test", base=repo.default_branch)


YOUR ENTIRE RESPONSE SHOULD BE EXCLUSIVELY PYTHON CODE. YOUR RESPONSE MUST RUN IN PYTHON AS-IS TO CREATE THE PULL REQUEST.

Your response will be run in a python script after the following code has already been run.

from github import Github, GithubException

# Login and get repo
g = Github(os.getenv("GH_TOKEN"))
repo = g.get_repo({repo_path})

"""

response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": first_message}
        {"role": "assistant", "content": first_response_content},
        # {"role": "user", "content": create_pr_message}
    ]
)

In [ ]:
## Need to separate out the middle of ```python\n XXXX \n```
## Probably need to specify to not import any non-standard packages
## Specifying that the response needs to be in python might be confusing if the repo is in another language
## Wrap calls to gpt-4 in try/except to handle connection issues

In [33]:
response

<OpenAIObject chat.completion id=chatcmpl-7LEQLiAyUo3fFtiACifZpZRTQawDJ at 0x129346330> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I understand the task and I'll provide you with the Python script and the modifications in the README.md you requested. \n\nHere's the Python script `repo_manager.py` to convert the code from the test.ipynb notebook into a command-line script:\n\n```python\nimport os\nimport argparse\nfrom github import Github, GithubException\n\ndef create_branch(repo, source_branch, target_branch):\n    try:\n        repo.get_branch(target_branch)\n    except GithubException as e:\n        if e.status == 404:\n            sb = repo.get_branch(source_branch)\n            repo.create_git_ref(ref='refs/heads/' + target_branch, sha=sb.commit.sha)\n\ndef create_file(repo, path, message, content, branch):\n    repo.create_file(path=path, message=message, content=content, branch=branch)\n\ndef update_file(

In [49]:
print("test")
test = input()

test


In [42]:
create_pr_response = response["choices"][0]["message"]["content"]

In [43]:
locs = parse_python_code(create_pr_response).split("\n")
for loc in locs:
    print(loc)

import os
import argparse
from github import Github, GithubException

def create_pull_request(repo, source_branch='main', target_branch='test'):
    try:
        repo.get_branch(target_branch)
    except GithubException as e:
        if e.status == 404:
            sb = repo.get_branch(source_branch)
            repo.create_git_ref(ref='refs/heads/' + target_branch, sha=sb.commit.sha)

    file_content = "This is the file content"
    commit_message = "create test file"
    repo.create_file(path="test.txt", message=commit_message, content=file_content, branch=target_branch)

    contents = repo.get_contents("test.txt", ref=target_branch)
    repo.update_file(contents.path, "more tests", "more tests", contents.sha, branch=target_branch)

    contents = repo.get_contents("test.txt", ref=target_branch)
    repo.delete_file(contents.path, message="remove test", sha=contents.sha, branch=target_branch)

    body = '''
    This PR created, updated and deleted a test file
    '''
    repo.crea

In [ ]:
exec(parse_python_code(create_pr_response))